In [1]:
import requests
import time
from bs4 import BeautifulSoup
import csv


In [2]:
def extract(jobtype,page=1):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}
    r = requests.get(url=f'https://builtin.com/jobs/remote?search={jobtype}&page={page}',headers=headers)
    if r.status_code == 200:
        print('success',page)
        return r.content
    else:
        print(f'status code {r.status_code} sorry :( failed on page {page}')
    return None

In [3]:
#extract job info from job cards and return a list of dictionarys with the title description salary and date posted
def pull_data(soup):
    data = []
    job_cards = soup.find_all('div', {'data-id': 'job-card'})
    for card in job_cards:

        job_info = {'title':'','description':'','salary':'','date_posted':''}
        job_info['title'] = card.find('div',{'data-id': 'company-title'}).text
        job_info['description'] = card.find('h2').text
        job_info['date_posted'] = card.find('span',string=lambda x: any(substring in x for substring in ['Ago','Yesterday'])).text

        pay_info = card.find('span', string=lambda x: 'Annually' in x)
        if pay_info:
            job_info['salary'] = pay_info.text

        data.append(job_info)
    return data

In [4]:
#find max page number for that jobtype change jobtype to search different types of jobs
jobtype = 'data'
html = extract(jobtype=jobtype)
soup = BeautifulSoup(html, 'html.parser')
max_page = soup.find_all('li', class_='mx-xs page-item')[-1].text

success 1


In [5]:
#pull data from all pages
data = []
for i in range(1,int(max_page)+1):
    html = extract(jobtype,i)
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        data.append(pull_data(soup))
    time.sleep(1)

success 1
success 2
success 3
success 4
success 5
success 6
success 7
success 8
success 9
success 10
success 11
success 12
success 13
success 14
success 15
success 16
success 17
success 18
success 19
success 20
success 21
success 22
success 23
success 24
success 25
success 26
success 27
success 28
success 29
success 30
success 31
success 32
success 33
success 34
success 35
success 36
success 37
success 38
success 39
success 40
success 41
success 42
success 43
success 44
success 45
success 46
success 47
success 48
success 49
success 50
success 51
success 52
success 53
success 54
success 55
success 56
success 57
success 58
success 59
success 60
success 61
success 62
success 63
success 64
success 65
success 66
success 67
success 68
success 69
success 70
success 71
success 72
success 73
success 74
success 75
success 76
success 77


In [6]:
#get rid of duplicates and merge lists to one
finalData = []
for page in data:
    for item in page:
        if item not in finalData:
            finalData.append(item)

In [7]:
headers = finalData[0].keys()
with open('builtInData.csv', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(finalData)